# EX2 : Proof of Work

                    - Implémentation de Proof of Work en Python, C ou C++.
                    - Une implémentation des blocs et chaines sera nécessaire.
                    - Changer à chaque fois la difficulté du hashage et calculer le temps d’exécution pour
                        chaque niveau de difficulté.
                    - Vérifiez le bon fonctionnement du concept en donnant des exemples d’exécution.

In [18]:
import hashlib

**defining a hash_data function for hashing**

In [13]:
def hash_data(data):
  h = hashlib.new('sha256')
  h.update(data.encode('utf-8'))
  return h.hexdigest()

*data.encode('utf-8'): Converts the input string to bytes, which is required by the hashing function.
hashlib.sha256(): Generates the SHA-256 hash of the input.
h.hexdigest(): Returns the hash as a hexadecimal string.*

**Node Class**
*The Node class represents each node in the Merkle Tree.*

In [14]:
# Node class that represents a node of a binary tree
class Node:
    def __init__(self, data):
        self.data = data  # Node data
        self.left = None  # Left child
        self.right = None  # Right child

    def isLeaf(self):
        # A node is a leaf if both left and right children are None
        return self.left is None and self.right is None

    def __str__(self):
        return self.data

**MerkleTree Class**
*This is where the core logic of the Merkle Tree is implemented.*
    

In [15]:
# Merkle tree class for actual implementation of the tree
class MerkleTree:
    def __init__(self):
        self.root = None  # Root of the tree
        self._merkleRoot = ''  # To store the final Merkle root hash

    # Function to create the Merkle tree from an array of transaction strings
    def makeTreeFromArray(self, arr):
        if len(arr) == 0:
            return

        # Initialize the list of leaf nodes with hashed transaction data
        leaves = [Node(hash_data(data)) for data in arr]

        # While more than one node remains, build the tree upwards
        while len(leaves) > 1:
            temp = []

            # Pair consecutive nodes and combine their hashes to create parent nodes
            for i in range(0, len(leaves), 2):
                left_child = leaves[i]
                if i + 1 < len(leaves):
                    right_child = leaves[i + 1]
                else:
                    # If there's an odd number of nodes, duplicate the last one
                    right_child = left_child

                # Create parent node by combining the left and right hashes
                combined_data = hash_data(left_child.data + right_child.data)
                parent = Node(combined_data)
                parent.left = left_child
                parent.right = right_child
                temp.append(parent)

            leaves = temp  # Continue to the next level with the parent nodes

        # The last remaining node is the root
        self.root = leaves[0]
        self._merkleRoot = self.root.data

    # Recursive inorder traversal utility to print the tree
    def inorderTraversal(self, node):
        if not node:
            return
        self.inorderTraversal(node.left)
        print(node)
        self.inorderTraversal(node.right)

    # Utility function to return the Merkle root
    def getMerkleRoot(self):
        return self._merkleRoot

    # Function to verify the integrity of a list of transactions
    def verify(self, arr1):
        original_root = self.getMerkleRoot()  # Get the root of the current tree

        # Create a new Merkle tree from the provided array and calculate its root
        new_tree = MerkleTree()
        new_tree.makeTreeFromArray(arr1)
        new_merkle_root = new_tree.getMerkleRoot()

        # Compare the original Merkle root with the new one
        if original_root == new_merkle_root:
            print("Transactions verified successfully.")
            return True
        else:
            print("Transactions have been tampered.")
            return False


**makeTreeFromArray(arr)**
        This function builds the Merkle Tree from an array of transaction strings.
        
        **Input:** arr is a list of transactions (e.g., ["tx1", "tx2", "tx3", "tx4"]).
        Process:
        1- Each transaction in the list is hashed using the hash_data function.
        2- The transactions are turned into leaf nodes.
        3- The parent nodes are created by combining the hashes of every two child nodes. The hash of the combined data becomes the parent’s hash.
        4- If there's an odd number of nodes at any level, the last node is duplicated to form a pair.
        5- This process repeats until there’s only one node left, which becomes the root of the tree.
        
![My Image](img.png)
      *H12 is the hash of H1 + H2, and H34 is the hash of H3 + H4.*

In [16]:
# Example usage
if __name__ == "__main__":
    # Create a list of transactions
    transactions = ["tx1", "tx2", "tx3", "tx4"]

    # Create a Merkle tree
    merkle_tree = MerkleTree()
    merkle_tree.makeTreeFromArray(transactions)

    # Print the Merkle root
    print("Merkle Root:", merkle_tree.getMerkleRoot())

    # Inorder traversal of the tree
    print("\nInorder Traversal of the Merkle Tree:")
    merkle_tree.inorderTraversal(merkle_tree.root)

    # Verification of the transactions
    print("\nVerifying transactions...")
    merkle_tree.verify(transactions)

    # Test with tampered transactions
    tampered_transactions = ["tx1", "tx2", "txX", "tx4"]
    print("\nVerifying tampered transactions...")
    merkle_tree.verify(tampered_transactions)

Merkle Root: 773bc304a3b0a626a520a8d6eacc36809ac18c0b174f3ff3cdaf0a4e9c64433d

Inorder Traversal of the Merkle Tree:
709b55bd3da0f5a838125bd0ee20c5bfdd7caba173912d4281cae816b79a201b
f8f28ede979567036d801ad6cf58b551c7d8530bba005c48e46d39c73ab52664
27ca64c092a959c7edc525ed45e845b1de6a7590d173fd2fad9133c8a779a1e3
773bc304a3b0a626a520a8d6eacc36809ac18c0b174f3ff3cdaf0a4e9c64433d
1f3cb18e896256d7d6bb8c11a6ec71f005c75de05e39beae5d93bbd1e2c8b7a9
850cf301915d09ebcfa84e2ee4087025e17a6fca7e4149ce02cff94cd3db55de
41b637cfd9eb3e2f60f734f9ca44e5c1559c6f481d49d6ed6891f3e9a086ac78

Verifying transactions...
Transactions verified successfully.

Verifying tampered transactions...
Transactions have been tampered.


**Building the Tree:**
The makeTreeFromArray() function creates the tree from the given transaction list.
**Getting the Merkle Root:**
The getMerkleRoot() function returns the hash of the root node, which is the Merkle Root.
**Inorder Traversal:**
The inorderTraversal() function prints out the tree in the inorder sequence (helpful for visualization).
**Verification:**
The verify() function checks if the transactions have been tampered with by comparing the Merkle Root of a new tree with the original one.

# Conclusion
**This code constructs a Merkle Tree from a list of transactions and verifies the integrity of the transactions using the Merkle Root. It's designed to detect any changes made to the transaction list by checking whether the newly calculated Merkle Root matches the original one.**